In [2]:
import numpy as np
from pandas import DataFrame
import keras
import pandas as pd

Using TensorFlow backend.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

# Récupération et mise en forme des données

In [ ]:
X = np.load('X.npy')
Y = np.load('Y.npy')

In [ ]:
X.shape

In [ ]:
X_flat = np.array(X).reshape((2062, 64*64))
X_flat.shape

In [ ]:
Ydf = DataFrame(data=Y, columns=[9-x for x in range(10)])
Ydf.head()

In [ ]:
Ydf = Ydf[[x for x in range(10)]]
Ydf.head()

In [ ]:
Ydf['class']=(Ydf[range(10)].idxmax(axis=1)).astype('int')
Ydf[['class']]

In [ ]:
Y_flat = Ydf['class'].to_numpy()
Y_flat.shape

In [ ]:
Y_flat = Y_flat.reshape(-1,1)
Y_flat.shape

In [ ]:
from numpy import random as np_random

SEED = 42
np_random.seed(SEED)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_flat, Y_flat, test_size=0.2)
X_train.shape

In [ ]:
X_val.shape

In [ ]:
y_train_df = DataFrame(y_train)
y_train_df.describe()

# Model NN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

n_input = X_train.shape[1]
n_input

In [ ]:
LEARNING_RATE = 0.001
BATCH_SIZE = n_input
EPOCHS = 13500

In [ ]:
model = Sequential([
    Input(shape=n_input),
    Dense(64, activation='relu',kernel_initializer='normal'),
    Dense(32, activation='relu',kernel_initializer='normal'),
    Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from keras import losses

# model.compile(loss='mse', optimizer=Adam(lr=LEARNING_RATE))
model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=LEARNING_RATE)) #RMSprop

In [ ]:
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_val, y_val),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose=2)

In [ ]:
pd.DataFrame(history.history).plot()
plt.grid(True)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

# Evaluate NN

In [ ]:
pred_val = model.predict(X_val)
pred_val

In [ ]:
df_pred_val = DataFrame(pred_val, columns=[x for x in range(10)])
df_pred_val.head()

In [ ]:
df_pred_val['class_pred']=(df_pred_val[range(10)].idxmax(axis=1)).astype('int')
df_pred_val.head()

In [ ]:
y_pred_val = df_pred_val['class_pred'].to_numpy()
y_pred_val = y_pred_val.reshape(-1,1)

In [ ]:
y_pred_val.shape

In [ ]:
y_val.shape

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val, y_pred_val)